In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

In [2]:
from datetime import date

In [3]:
USERNAME = "SumanjaliDamarla"
PASSWORD = "D.Sumanjali1"
API_SOURCE = "https://scihub.copernicus.eu/dhus"

In [4]:
#download for different months and 2 areas, then check for processing level and cloud data
STUDY_AREA = "StudyArea2.geojson"
SATELLITE = "Sentinel-2"
PROCESSING_LEVEL = ["Level-1", "Level-2"]

In [5]:
api = SentinelAPI( USERNAME, PASSWORD, API_SOURCE)

In [6]:
footprint = geojson_to_wkt(read_geojson(STUDY_AREA))
footprint

'MULTIPOLYGON(((81.1080 16.7753,81.6577 16.9750,82.4383 17.2674,82.4779 17.2384,82.4838 17.0470,82.3035 16.5313,81.5531 16.1905,81.1327 16.6796,81.1080 16.7753)))'

In [7]:
NOV_BEGIN = "20181101"
NOV_END = "20181130"

DEC_BEGIN = "20181201"
DEC_END = "20181231"

JAN_BEGIN = "20190101"
JAN_END = "20190131"

FEB_BEGIN = "20190201"
FEB_END = "20190228"

MAR_BEGIN = "20190301"
MAR_END = "20190331"

In [7]:
type(footprint)

str

In [8]:
def queryForProducts(footprint, beg, end, sat):
    return api.query(footprint, date=(beg, end),platformname=sat)

In [9]:
def getLevelURLs(df):
    level1 = []
    for i in range(df.shape[0]):
        if PROCESSING_LEVEL[0] in df["processinglevel"][i]:
            level1.append(df["link"][i])

    level2 = []
    for i in range(df.shape[0]):
        if PROCESSING_LEVEL[1] in df["processinglevel"][i]:
            level2.append(df["link"][i])
    
    return level1, level2

In [10]:
nov_products = queryForProducts(footprint,NOV_BEGIN, NOV_END,SATELLITE)
dec_products = queryForProducts(footprint,DEC_BEGIN, DEC_END,SATELLITE)
jan_products = queryForProducts(footprint,JAN_BEGIN, JAN_END,SATELLITE)
feb_products = queryForProducts(footprint,FEB_BEGIN, FEB_END,SATELLITE)
mar_products = queryForProducts(footprint,MAR_BEGIN, MAR_END,SATELLITE)

In [11]:
nov_df = api.to_dataframe(nov_products)
dec_df = api.to_dataframe(dec_products)
jan_df = api.to_dataframe(jan_products)
feb_df = api.to_dataframe(feb_products)
mar_df = api.to_dataframe(mar_products)

In [12]:
nov_level1, nov_level2 = getLevelURLs(nov_df)
dec_level1, dec_level2 = getLevelURLs(dec_df)
jan_level1, jan_level2 = getLevelURLs(jan_df)
feb_level1, feb_level2 = getLevelURLs(feb_df)
mar_level1, mar_level2 = getLevelURLs(mar_df)

In [13]:
months = [
    "Nov", "Dec", "Jan", "Feb", "Mar"
]
levelCount = [
    [nov_df.shape[0], len(nov_level1), len(nov_level2)],
    [dec_df.shape[0], len(dec_level1), len(dec_level2)],
    [jan_df.shape[0], len(jan_level1), len(jan_level2)],
    [feb_df.shape[0], len(feb_level1), len(feb_level2)],
    [mar_df.shape[0], len(mar_level1), len(mar_level2)]
]

print("      Total  L1  L2")
for i in range(len(months)):
    print(months[i] + "   " + str(levelCount[i][0]) + "     " + str(levelCount[i][1]) + "  " + str(levelCount[i][2]))

      Total  L1  L2
Nov   31     31  0
Dec   47     32  15
Jan   60     30  30
Feb   52     26  26
Mar   60     30  30


In [1]:
#download using processes
#arrange using filenames
#use loops to perform tasks for all months together
#use print and return statements in functions to show the processes
#refactor the code
#clean code
#delete unnecessary things
#mail
#github
#update
#stacking, mosaic and clip using QGIS